In [3]:
import pandas as pd
import numpy as np
from datetime import date
from dateutil.relativedelta import relativedelta

from pydwh import DBConnection

In [2]:
#connection to DWH
dwh = DBConnection()

In [37]:
#connection to SWH
swh = DBConnection(db='segment_warehouse')

In [3]:
path = '/Users/catriona/email_evaluation/layouts_camp_mar20/'

In [17]:
form_info = '''select form_id, form_uid, full_date, plan, met_tot_unique_submissions, account_creation_date,
(case when datediff(day, account_creation_date, full_date)<= 30 and plan in ('core','Basic') then 'New Free'
when datediff(day, account_creation_date, full_date)<= 30 and plan not in ('core','Basic') then 'New Pay'
when plan in ('core','Basic') then 'Existing Free' else 'Existing Pay' end) as cust_type, days_since
from rpt.form_metrics_daily a inner join rpt.qualified_users b on (a.account_id=b.account_id)
where full_date between '2019-10-01' and '2020-01-01'
'''
form_info = dwh.sql_query_to_data_frame(form_info)

In [18]:
form_info = form_info.sort_values(by=['form_uid','full_date'], ascending=True).reset_index(drop=True)
form_info['cum_submissions'] = form_info.groupby(['form_uid'])['met_tot_unique_submissions'].cumsum()
form_info.head(10)

,form_id,form_uid,full_date,plan,met_tot_unique_submissions,account_creation_date,cust_type,days_since,cum_submissions
0,15052173,A00CZS,2019-10-09,core,7,2019-01-23 00:48:05,Existing Free,0,7
1,15052173,A00CZS,2019-10-09,core,2,2019-01-23 00:48:05,Existing Free,0,9
2,15052173,A00CZS,2019-10-10,core,1,2019-01-23 00:48:05,Existing Free,1,10
3,15052173,A00CZS,2019-10-10,core,2,2019-01-23 00:48:05,Existing Free,1,12
4,15052173,A00CZS,2019-10-11,core,1,2019-01-23 00:48:05,Existing Free,2,13
5,15052173,A00CZS,2019-10-15,core,2,2019-01-23 00:48:05,Existing Free,6,15
6,15052173,A00CZS,2019-10-15,core,1,2019-01-23 00:48:05,Existing Free,6,16
7,15052173,A00CZS,2019-10-17,core,1,2019-01-23 00:48:05,Existing Free,8,17
8,15052173,A00CZS,2019-10-18,core,0,2019-01-23 00:48:05,Existing Free,9,17
9,15052173,A00CZS,2019-10-20,core,1,2019-01-23 00:48:05,Existing Free,11,18


In [25]:
form_created = form_info[form_info['days_since'] == 0]
form_created = form_created.drop_duplicates(subset='form_id', keep='first')
form_created = form_created[['form_id','form_uid','full_date','cust_type']]
form_created = form_created.rename(columns={'full_date':'form_created'})

form_activated = form_info[form_info['cum_submissions'] >= 5]
form_activated = form_activated.drop_duplicates(subset='form_id', keep='first')
form_activated = form_activated[['form_uid','full_date','days_since']]
form_activated = form_activated.rename(columns={'full_date':'form_activated'})

In [26]:
form_act = pd.merge(form_created,form_activated, how='left', on='form_uid')
form_act.head(10)

,form_id,form_uid,form_created,cust_type,form_activated,days_since
0,15052173,A00CZS,2019-10-09,Existing Free,2019-10-09,0.0
1,15668628,A00OsP,2019-12-02,Existing Pay,NaT,NaN
2,15217269,A00c9W,2019-10-23,Existing Pay,2019-11-08,16.0
3,15619985,A0206a,2019-11-26,Existing Pay,NaT,NaN
4,15204099,A03Igk,2019-10-22,New Free,NaT,NaN
5,15109212,A03Lv2,2019-10-14,New Free,NaT,NaN
6,15462650,A05AVe,2019-11-13,Existing Pay,2019-11-20,7.0
7,15438568,A05CT4,2019-11-11,Existing Pay,NaT,NaN
8,15008869,A05s6k,2019-10-04,New Free,2019-10-05,1.0
9,15520688,A07WyK,2019-11-18,New Free,2019-11-18,0.0


In [27]:
not_active = form_act[form_act['days_since'].isna()]

In [33]:
active = form_act[~form_act['days_since'].isna()]
active.loc[:,'days_between'] = (active['form_activated'] - active['form_created']).dt.days

In [34]:
active.head()

,form_id,form_uid,form_created,cust_type,form_activated,days_since,days_between
0,15052173,A00CZS,2019-10-09,Existing Free,2019-10-09,0.0,0
2,15217269,A00c9W,2019-10-23,Existing Pay,2019-11-08,16.0,16
6,15462650,A05AVe,2019-11-13,Existing Pay,2019-11-20,7.0,7
8,15008869,A05s6k,2019-10-04,New Free,2019-10-05,1.0,1
9,15520688,A07WyK,2019-11-18,New Free,2019-11-18,0.0,0


In [35]:
not_active['cust_type'].value_counts()

Existing Pay     69970
New Free         32663
Existing Free    18304
New Pay           5041
Name: cust_type, dtype: int64

In [38]:
active.groupby('cust_type').agg({'form_id':'count','days_between':['mean','max']})

form_id days_between    
                count         mean max
cust_type                             
Existing Free   23008     4.209927  92
Existing Pay    61360     4.913885  88
New Free        22985     3.840766  91
New Pay          5978     5.537638  78

In [42]:
cust_type = active.groupby('cust_type')
cust_type['days_between'].describe(percentiles= [ 0.25, 0.5, 0.75, 0.85, 0.90, 0.95, 0.98 ])

,count,mean,std,min,25%,50%,75%,85%,90%,95%,98%,max
cust_type,,,,,,,,,,,,
Existing Free,23008.0,4.209927,8.503636,0.0,0.0,1.0,4.0,8.0,12.0,21.0,34.00,92.0
Existing Pay,61360.0,4.913885,8.961990,0.0,0.0,1.0,6.0,10.0,14.0,23.0,36.00,88.0
New Free,22985.0,3.840766,8.108254,0.0,0.0,1.0,4.0,7.0,11.0,20.0,31.32,91.0
New Pay,5978.0,5.537638,9.571758,0.0,0.0,2.0,6.0,11.0,16.0,25.0,38.00,78.0


In [8]:
def dt_range(start, end, step):
    while pd.to_datetime(start) <= pd.to_datetime(end):
        yield pd.to_datetime(start)
        start = pd.to_datetime(start) + pd.DateOffset(days=(step+1))

In [38]:
df_out = pd.DataFrame()

In [ ]:
import datetime
def dt_range(start, end, step):
    while pd.to_datetime(start) <= pd.to_datetime(end):
        yield pd.to_datetime(start)
        
        pd_st = pd.to_datetime(start)
        pd_ed = pd.to_datetime(start) + pd.DateOffset(days=step)
        
        form_query1 = ''' select account_id, form_uid, {0} as pd_st, {1} as pd_ed, (case when min_days_since = 0 then 1 else 0 end) as created_in_per,
        (case when tot_subs >= 1 then 1 else 0 end) as collecting_in_per, (case when uniq_subs >= 5 then 1 else 0 end) as active_in_per,
        (case when account_creation_date between {0} and {1} and plan in ('core') then 'New Free'
        case when account_creation_date between {0} and {1} and plan not in ('core') then 'New Paying'
        case when plan in ('core') then 'Existing Free' case when plan not in ('core') then 'Existing Paying' else 'NA' end) as cust_type,
        (case when min_days_since = 0 and uniq_subs >= 5 then 1 else 0 end) as created_active_forms
        from
        (select account_id, form_uid, plan, min(days_since) as min_days_since, sum(met_tot_submissions) as tot_subs, sum(met_tot_unique_submissions) as uniq_subs
        from rpt.form_metrics_daily where full_date between {0} and {1}
        group by account_id, form_uid, plan) a left join dwh.dim_account b on (a.account_id=b.account_id)
        '''.format(pd_st, pd_ed)

        form_query2 = '''select form_uid, {0} as pd_st, {1} as pd_ed, 1 as any_media_feat, 
        max(case when media_features = 'Icons' then 1 else 0 end) as icons,
        max(case when media_features = 'Images' then 1 else 0 end) as images,
        max(case when media_features = 'videos' then 1 else 0 end) as videos,
        max(case when media_features = 'layouts' then 1 else 0 end) as layouts,
        max(case when media_features = 'preview' then 1 else 0 end) as preview,
        max(case when media_features = 'logic_jump' then 1 else 0 end) as logic_jump
        from
        (select account_id, uuid, user_id, substring(context_page_path,7, position('/' in substring(context_page_path,7))-1) as form_uid,
         (case when feature = 'add_image' and image_source = 'icons' then 'Icons'
         when feature = 'add_image' and image_source != 'icons' then 'Images'
         when feature = 'add_video' then 'videos'
         when feature = 'layouts' then 'layouts'
         when feature = 'live_preview' then 'preview'
         when feature in ('logic-jump','logic_jump_preview') then 'logic_jump' else 'NA' end) as media_features
         from prod.use_feature
        where timestamp between {0} and {1} and feature in ('add_image','add_video','layouts','live_preview','logic-jump','logic_jump_preview') )
        group by form_uid
        '''.format(pd_st, pd_ed)
        
        forms1 = dwh.sql_query_to_data_frame(form_query1)
        forms2 = swh.sql_query_to_data_frame(form_query2)
        form_info = pd.merge(forms1, forms2, on=['form_uid','pd_st','pd_ed'], how='left')
        form_info['created_with_any_media'] = np.where((form_info['created_in_per']==1)&(form_info['any_media_feat']==1),1,0)
        form_info['created_with_any_icons'] = np.where((form_info['created_in_per']==1)&(form_info['icons']==1),1,0)
        form_info['created_with_any_images'] = np.where((form_info['created_in_per']==1)&(form_info['images']==1),1,0)
        form_info['created_with_any_videos'] = np.where((form_info['created_in_per']==1)&(form_info['videos']==1),1,0)
        form_info['created_with_any_layouts'] = np.where((form_info['created_in_per']==1)&(form_info['layouts']==1),1,0)
        form_info['created_with_any_preview'] = np.where((form_info['created_in_per']==1)&(form_info['preview']==1),1,0)
        form_info['created_with_any_logicj'] = np.where((form_info['created_in_per']==1)&(form_info['logic_jump']==1),1,0)
        
        form_info['activated_with_any_media'] = np.where((form_info['active_in_per']==1)&(form_info['any_media_feat']==1),1,0)
        form_info['activated_with_any_icons'] = np.where((form_info['active_in_per']==1)&(form_info['icons']==1),1,0)
        form_info['activated_with_any_images'] = np.where((form_info['active_in_per']==1)&(form_info['images']==1),1,0)
        form_info['activated_with_any_videos'] = np.where((form_info['active_in_per']==1)&(form_info['videos']==1),1,0)
        form_info['activated_with_any_layouts'] = np.where((form_info['active_in_per']==1)&(form_info['layouts']==1),1,0)
        form_info['activated_with_any_preview'] = np.where((form_info['active_in_per']==1)&(form_info['preview']==1),1,0)
        form_info['activated_with_any_logicj'] = np.where((form_info['active_in_per']==1)&(form_info['logic_jump']==1),1,0)
        
        form_info['create_act_with_any_media'] = np.where((form_info['created_active_forms']==1)&(form_info['any_media_feat']==1),1,0)
        form_info['create_act_with_any_icons'] = np.where((form_info['created_active_forms']==1)&(form_info['icons']==1),1,0)
        form_info['create_act_with_any_images'] = np.where((form_info['created_active_forms']==1)&(form_info['images']==1),1,0)
        form_info['create_act_with_any_videos'] = np.where((form_info['created_active_forms']==1)&(form_info['videos']==1),1,0)
        form_info['create_act_with_any_layouts'] = np.where((form_info['created_active_forms']==1)&(form_info['layouts']==1),1,0)
        form_info['create_act_with_any_preview'] = np.where((form_info['created_active_forms']==1)&(form_info['preview']==1),1,0)
        form_info['create_act_with_any_logicj'] = np.where((form_info['created_active_forms']==1)&(form_info['logic_jump']==1),1,0)
        
        account_forms = forms_info.groupby(['account_id','cust_type','pd_st']).agg({'created_in_per':'sum','active_in_per':'sum','created_active_forms':'sum'
                                                                                   'created_with_any_media':'sum','created_with_any_icons':'sum','created_with_any_images':'sum',
                                                                                   'created_with_any_videos':'sum','created_with_any_layouts':'sum','created_with_any_preview':'sum',
                                                                                   'created_with_any_logicj':'sum','activated_with_any_media':'sum','activated_with_any_icons':'sum',
                                                                                   'activated_with_any_images':'sum','activated_with_any_videos':'sum','activated_with_any_layouts':'sum',
                                                                                   'activated_with_any_preview':'sum','activated_with_any_logicj':'sum','create_act_with_any_media':'sum',
                                                                                   'create_act_with_any_icons':'sum','create_act_with_any_images':'sum','create_act_with_any_videos':'sum',
                                                                                   'create_act_with_any_layouts':'sum','create_act_with_any_preview':'sum','create_act_with_any_logicj':'sum'}).reset_index()
        
        channel = '''select account_id, {0} as pd_st, max(case when landing_page = 'https://www.typeform.com/' then 1 else 0 end) as homepage,
        max(case when landing_page = 'workspace' then 1 else 0 end) as workspace,
        max(case when landing_page like '%blog%' then 1 else 0 end) as blog,
        max(case when landing_page like '%template%' then 1 else 0 end) as template_gal,
        max(case when main_channel = 'social' then 1 else 0 end) as social,
        max(case when main_channel = 'email' then 1 else 0 end) as email,
        max(case when main_channel = 'paid_social' then 1 else 0 end) as paid_social,
        max(case when main_channel = 'referral' then 1 else 0 end) as referral
        from attribution.attribution_pre_model
        where event_date between {0} and {1}
        group by account_id
        '''.format(pd_st, pd_ed)
        
        accounts = '''select a.account_id, {0} as pd_st, max(case when a.created_at between {0} and {1} then 1 else 0 end) as signup,
        max(case when a.verified_at between {0} and {1} then 1 else 0 end) as verified,
        max(case when b.first_form_created between {0} and {1} then 1 else 0 end) as create_first_form,
        max(case when b.account_qualification_date between {0} and {1} then 1 else 0 end) as qualified,
        max(case when b.account_activation_date between {0} and {1} then 1 else 0 end) as activated,
        max(case when b.account_conversion_date between {0} and {1} then 1 else 0 end) as converted,
        max(case when c.event='unsubscribe' and c.event_created between {0} and {1} then 1 else 0 end) as unsubscribe
        max(case when a.deleted_at between {0} and {1} then 1 else 0 end) as account_deleted
       
        from dwh.dim_account a left join rpt.qualified_users b on (a.account_id=b.account_id)
        left join rpt.stripe_subscription_events c on (a.account_id=c.account_id)
        where a.created_at <= {1} and a.email not like '%@typeform.com'
        group by a.account_id
        '''.format(pd_st, pd_ed)
        
        account_all = pd.merge(accounts, account_forms, on=['account_id','pd_st'], how='left')
        account_all = pd.merge(account_all, account_forms, on=['account_id','pd_st'], how='left')
        
        account_summ = account_all.groupby(['pd_st','cust_type']).agg({'account_id':'count',})
        
        df_out = df_out.append(account_all)
        start = pd.to_datetime(start) + pd.DateOffset(days=(step+1))
        
        return df_out



In [10]:
dt_list = []
for x in dt_range('01-01-2019', '30-06-2019', 28):
    dt_list.append(x)  

In [11]:
dt_list

[Timestamp('2019-01-01 00:00:00'),
 Timestamp('2019-01-30 00:00:00'),
 Timestamp('2019-02-28 00:00:00'),
 Timestamp('2019-03-29 00:00:00'),
 Timestamp('2019-04-27 00:00:00'),
 Timestamp('2019-05-26 00:00:00'),
 Timestamp('2019-06-24 00:00:00')]

In [13]:
for x in dt_list:
    x2 = pd.to_datetime(x) + pd.DateOffset(days=(28))
    print(x)
    print(x2)

2019-01-01 00:00:00
2019-01-29 00:00:00
2019-01-30 00:00:00
2019-02-27 00:00:00
2019-02-28 00:00:00
2019-03-28 00:00:00
2019-03-29 00:00:00
2019-04-26 00:00:00
2019-04-27 00:00:00
2019-05-25 00:00:00
2019-05-26 00:00:00
2019-06-23 00:00:00
2019-06-24 00:00:00
2019-07-22 00:00:00
